In [1]:
import os
os.environ["HTTPS_PROXY"] = ""
os.environ["HTTP_PROXY"] = ""

In [2]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

import warnings
warnings.filterwarnings('ignore')

In [3]:
path = "Analytics/Customer Projects/Kohls - proj 11696 (Custom RV append, Fraud Intel)/File from the Customer/LNRV_RETROTHICK.csv.gpg.out"

In [4]:
from dsgtools import azure
data = pd.read_adls(path, reader = pd.read_csv, dtype = str, encoding='iso-8859-1')
print(data.shape)

(4653117, 64)


In [5]:
data.app_num.is_unique

False

In [6]:
data = data.sort_values(by = ["app_num", "wof_cnt", "wof_amt_sum", "bal_sum"])
data = data.drop_duplicates(subset = ["app_num"], keep = "last")
print(data.shape)
data.app_num.is_unique

(4653113, 64)


True

## Overall File

In [7]:
data[['CONSUMER_FIRST_NAME', 'CONSUMER_MIDDLE_NAME', 'CONSUMER_LAST_NAME', 'input_ADDR_LN1', 'input_ADDR_LN2', 'input_CITY', 'input_state_abbr', 
'input_POSTAL_CODE', 'INITIAL_APPLICATION_SSN', 'birth_dt', 'CONSUMER_HOME_PHONE']].isnull().sum()

CONSUMER_FIRST_NAME            265
CONSUMER_MIDDLE_NAME       4398279
CONSUMER_LAST_NAME             347
input_ADDR_LN1                 351
input_ADDR_LN2             4653110
input_CITY                 1542330
input_state_abbr           1542330
input_POSTAL_CODE                3
INITIAL_APPLICATION_SSN        576
birth_dt                         0
CONSUMER_HOME_PHONE            239
dtype: int64

In [8]:
data["address"] = np.where(data.input_ADDR_LN1.notnull(), np.where(data.input_ADDR_LN2.notnull(), data.input_ADDR_LN1 + " " + data.input_ADDR_LN2, data.input_ADDR_LN1), data.input_ADDR_LN1)
Input_FirstName = "CONSUMER_FIRST_NAME"
Input_LastName = "CONSUMER_LAST_NAME"
Input_StreetAddress = "address"
Input_City = "input_CITY"
Input_State = "input_state_abbr"
Input_Zip = "input_POSTAL_CODE"
Input_SSN = "INITIAL_APPLICATION_SSN"

data["sufficient_input"] = np.where((data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_StreetAddress].notnull() & data[Input_Zip].notnull()) | \
             (data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_StreetAddress].notnull() & data[Input_City].notnull() & data[Input_State].notnull()) | \
             (data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_SSN].notnull()), 1, 0)
freq(data["sufficient_input"])

,Count,Pct,Cuml Count,Cuml Pct
sufficient_input,,,,
0,374.0,0.00008,374.0,0.00008
1,4652739.0,0.99992,4653113.0,1.00000


In [9]:
data = data[data["sufficient_input"] == 1]
from datetime import datetime
data["APPDATETIME_fmt"] = pd.to_datetime(data.app_dt.str.slice(0,9))
data["APPDATETIME_fmt_F"] = data['APPDATETIME_fmt'].astype(str).str.slice(0,4) + data['APPDATETIME_fmt'].astype(str).str.slice(5,7)
data["APPDATETIME_fmt_F_dt"] = data['APPDATETIME_fmt'].astype(str).str.slice(0,4) + data['APPDATETIME_fmt'].astype(str).str.slice(5,7) + data['APPDATETIME_fmt'].astype(str).str.slice(8,10)
freq(data["APPDATETIME_fmt_F"] )

,Count,Pct,Cuml Count,Cuml Pct
APPDATETIME_fmt_F,,,,
201901,259605.0,0.055796,259605.0,0.055796
201905,426469.0,0.091660,686074.0,0.147456
201908,459708.0,0.098804,1145782.0,0.246260
201911,607933.0,0.130661,1753715.0,0.376921
201912,659956.0,0.141842,2413671.0,0.518763
202101,265659.0,0.057097,2679330.0,0.575861
202105,448145.0,0.096319,3127475.0,0.672179
202108,404884.0,0.087021,3532359.0,0.759200
202111,573880.0,0.123342,4106239.0,0.882542


In [10]:
data["ReBuilt_wof_flag"] = np.where((data.FINAL_DECISION == "N") | (data.wof_cnt.isnull()), -1, np.where(data.wof_cnt.astype(float)  > 0, 1, 0))
freq(data.ReBuilt_wof_flag)

,Count,Pct,Cuml Count,Cuml Pct
ReBuilt_wof_flag,,,,
-1,2004166.0,0.430750,2004166.0,0.430750
0,2584333.0,0.555443,4588499.0,0.986193
1,64240.0,0.013807,4652739.0,1.000000


In [11]:
bivariate("APPDATETIME_fmt_F", "ReBuilt_wof_flag", df = data[data.ReBuilt_wof_flag !=-1])

tag               ReBuilt_wof_flag                                         \
stats                            N      PctN      Sum      Mean       WoE   
APPDATETIME_fmt_F                                                           
201901                    145509.0  0.054939   4269.0  0.029338  0.195508   
201905                    247112.0  0.093300   6340.0  0.025656  0.057617   
201908                    257877.0  0.097365   6713.0  0.026032  0.072529   
201911                    333841.0  0.126046   6882.0  0.020615 -0.166336   
201912                    372114.0  0.140496   6974.0  0.018742 -0.263503   
202101                    132595.0  0.050063   1644.0  0.012399 -0.683102   
202105                    258855.0  0.097734   5175.0  0.019992 -0.197645   
202108                    235521.0  0.088924   6705.0  0.028469  0.164524   
202111                    345920.0  0.130606   9349.0  0.027026  0.111049   
202112                    319229.0  0.120529  10189.0  0.031918  0.282427   
Missing                        0.0  0.000000      0.0  0.000000  0.000000   
Total                    2648573.0  1.000000  64240.0  0.024255  0.000000   

tag                          
stats                    IV  
APPDATETIME_fmt_F            
201901             0.002307  
201905             0.000318  
201908             0.000530  
201911             0.003225  
201912             0.008624  
202101             0.017132  
202105             0.003479  
202108             0.002605  
202111             0.001699  
202112             0.011022  
Missing            0.000000  
Total              0.050942

In [12]:
data = data.sort_values(by = ['CONSUMER_FIRST_NAME', 'CONSUMER_MIDDLE_NAME', 'CONSUMER_LAST_NAME', 'address', 'input_CITY', 'input_state_abbr', 
'input_POSTAL_CODE', 'INITIAL_APPLICATION_SSN', 'birth_dt', 'CONSUMER_HOME_PHONE', 'APPDATETIME_fmt_F_dt', 'ReBuilt_wof_flag'], ascending = False)
print(data.shape)
data = data.drop_duplicates(subset = ['CONSUMER_FIRST_NAME', 'CONSUMER_MIDDLE_NAME', 'CONSUMER_LAST_NAME', 'address', 'input_CITY', 'input_state_abbr', 
'input_POSTAL_CODE', 'INITIAL_APPLICATION_SSN', 'birth_dt', 'CONSUMER_HOME_PHONE', 'APPDATETIME_fmt_F_dt',], keep = "first")
print(data.shape)

(4652739, 70)
(4652190, 70)


In [12]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/Full_sample_formatted_w_flags/ready_to_be_formatted/kohls_11696_full_file.parquet",
             overwrite = True)

In [3]:
from dsgtools import azure
data = pd.read_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/Full_sample_formatted_w_flags/ready_to_be_formatted/kohls_11696_full_file.parquet",)
print(data.shape)

(4652190, 70)


In [9]:
from datetime import datetime
data["birth_dt_fmt"] = pd.to_datetime(data.birth_dt.str.slice(0,9), errors = 'coerce', format = "%d%b%Y")
data["birth_dt_fmt_dt"] = data['birth_dt_fmt'].astype(str).str.slice(0,4) + data['birth_dt_fmt'].astype(str).str.slice(5,7) + data['birth_dt_fmt'].astype(str).str.slice(8,10)

In [11]:
data[['birth_dt', "birth_dt_fmt", "birth_dt_fmt_dt"]].head()

,birth_dt,birth_dt_fmt,birth_dt_fmt_dt
3704952,04NOV1985,1985-11-04,19851104
3834873,24NOV1995,1995-11-24,19951124
4338931,24NOV1995,1995-11-24,19951124
1980991,10FEB1999,1999-02-10,19990210
161496,16AUG1992,1992-08-16,19920816


## Sample for validation

In [13]:
freq(data.FINAL_DECISION)

,Count,Pct,Cuml Count,Cuml Pct
FINAL_DECISION,,,,
N,2003599.0,0.430679,2003599.0,0.430679
Y,2648591.0,0.569321,4652190.0,1.000000


In [14]:
data = data[data.FINAL_DECISION == "Y"]
print(data.shape)
freq(data.fraud_type)

(2648591, 72)


,Count,Pct,Cuml Count,Cuml Pct
fraud_type,,,,
Authorized User,1.0,3.775592e-07,1.0,3.775592e-07
Family Fraud,4.0,1.510237e-06,5.0,1.887796e-06
ID Theft,115.0,4.341931e-05,120.0,4.530711e-05
Non-ID Theft/Shopping Acct Takeover,1145.0,4.323053e-04,1265.0,4.776124e-04
Online ID Theft,2438.0,9.204894e-04,3703.0,1.398102e-03
Payment Fraud,987.0,3.726510e-04,4690.0,1.770753e-03
Stolen Card,2255.0,8.513961e-04,6945.0,2.622149e-03
Stolen Info,1191.0,4.496731e-04,8136.0,3.071822e-03
Store ID Theft,2625.0,9.910930e-04,10761.0,4.062915e-03


In [15]:
data = data[data.fraud_type.isnull()]
freq(data.wof_cnt, data.ReBuilt_wof_flag, cross = False, observed = True)

,,Count,Pct,Cuml Count,Cuml Pct
wof_cnt,ReBuilt_wof_flag,,,,
0,0,2574308.0,9.763660e-01,2574308.0,0.976366
1,1,62247.0,2.360862e-02,2636555.0,0.999975
Missing,-1,65.0,2.465276e-05,2636620.0,0.999999
2,1,2.0,7.585464e-07,2636622.0,1.000000


In [16]:
data = data[data.wof_cnt.notnull()]
bivariate("APPDATETIME_fmt_F", "ReBuilt_wof_flag", df = data)

tag               ReBuilt_wof_flag                                         \
stats                            N      PctN      Sum      Mean       WoE   
APPDATETIME_fmt_F                                                           
201901                    144345.0  0.054748   4017.0  0.027829  0.168739   
201905                    245954.0  0.093286   6107.0  0.024830  0.051620   
201908                    256638.0  0.097338   6460.0  0.025172  0.065642   
201911                    332293.0  0.126033   6599.0  0.019859 -0.176854   
201912                    370469.0  0.140512   6715.0  0.018126 -0.269948   
202101                    132027.0  0.050076   1591.0  0.012051 -0.684334   
202105                    257902.0  0.097818   5045.0  0.019562 -0.192241   
202108                    234475.0  0.088932   6550.0  0.027935  0.172634   
202111                    344559.0  0.130685   9160.0  0.026585  0.121711   
202112                    317895.0  0.120572  10005.0  0.031473  0.295528   
Missing                        0.0  0.000000      0.0  0.000000  0.000000   
Total                    2636557.0  1.000000  62249.0  0.023610  0.000000   

tag                          
stats                    IV  
APPDATETIME_fmt_F            
201901             0.001691  
201905             0.000255  
201908             0.000433  
201911             0.003627  
201912             0.009024  
202101             0.017184  
202105             0.003302  
202108             0.002880  
202111             0.002052  
202112             0.012153  
Missing            0.000000  
Total              0.052601

In [17]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/cleaned/kohls_cleaned_11_21.parquet", overwrite = True)

### Add Weight

In [18]:
data["weight"] = 1
data_1 = data[(data.ReBuilt_wof_flag == 1)].copy()
data_0 = data[(data.ReBuilt_wof_flag == 0)].copy()
print(data_1.shape)
print(data_0.shape)

(62249, 73)
(2574308, 73)


In [19]:
freq(data.ReBuilt_wof_flag)

,Count,Pct,Cuml Count,Cuml Pct
ReBuilt_wof_flag,,,,
0,2574308.0,0.97639,2574308.0,0.97639
1,62249.0,0.02361,2636557.0,1.00000


In [20]:
data_0_sample = data_0.sample(n = 250_000 - data_1.shape[0], random_state=1)
data_0_sample['weight'] = data_0.shape[0]/(250_000 - data_1.shape[0])
print(data_0_sample['weight'].unique())
print(data_0_sample.shape)

[13.71128782]
(187751, 73)


In [21]:
data = pd.concat([data_1, data_0_sample], ignore_index= True)
print(data.shape)

(250000, 73)


In [22]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/kohls_250K_11_21.parquet", overwrite = True)